# Lab 3

## Lab Purpose

This lab continues our work on the syntax and functions of the python language. It also integrates this work with the means of automating ArcGIS desktop (and some server) through the ArcPy site package. Later on in the course we'll discuss ArcPro and the ArcGIS API for python, but - _in this lab_ - we focus on the more established means of automation.

We are going to explore how to search, query, and (somewhat) visualize spatial data in ArcPy.

Along the way, I'll ask you some (seemingly) goofy questions that will help build your familiarity with loops, flow control, and iteration.

### Question 1 - All Lovecraft all the time

First, let’s return to our old friend H.P. Lovecraft and his The Shunned House.

Last time, we counted the words he loved to use and how he used them. But, Lovecraft is also known for his lugubrious sentences, the long-winding means by which he tells us his terrible tales. I’m interested in how many characters his average sentence is. I’d also like a printed copy of the longest sentence and the shortest sentence in The Shunned House.

Think about how you opened the file, read it, and parsed it into a list last time… _What means do you have to count the character length of objects? What format do those objects have to be? Etc._

This question will help you think about ways to parse through and iterate over different types of data sets (in this case a text file).

In [ ]:
import re

# open file
shunned = open("shunned_house.txt").read()

# cleanup some stuff in the text file before processing
replaceList = {"*":"", 
               "_":"",
               "\r":"", 
               "\n":" ", 
               "[":"",
               "]":"",
               "a. m.":"am",
               "p. m.":"pm"
              }

for item in replaceList:
    shunned = shunned.replace(item, replaceList[item])

# split on spaces after punctuation unless it's for an abbreviation/initial in a name
# (may not catch all of them)
shunnedList = re.split("(?<!Mr\.|Dr\.|Ms\.|St\.)" +
                       "(?<!Mrs\.)" +
                       "(?<!\s[A-Z]\.)" +
                       "(?<=[.!?])" +
                       "\"?\s+", 
                       shunned)

# reassemble a copy of the cleaned text to get a character count for the avg
shunnedCleaned = "".join(shunnedList)

# collect character counts for each sentance
# format [[<char count>,<sentence>], ...]
shunnedCount = []

for item in shunnedList:
    shunnedCount.append([len(item), item])

# sort the list so the sentances of interest at are at either end of the list
shunnedCount = sorted(shunnedCount) 

# calulate & print average: total chars / number of sentances
avgChars = str(len(shunnedCleaned)/len(shunnedList))
print("Avg char/sentance: {0}\n".format(avgChars))

# print longest and shortest sentances
print("Longest Sentance, {0} characters:\n{1}\n".format(str(shunnedCount[-1][0]), shunnedCount[-1][1]))
print("Shortest Sentance, {0} characters:\n{1}".format(str(shunnedCount[0][0]), shunnedCount[0][1]))

### Question 2 - An Odd Request (part 1)

Having handled text, now let's take a look at how we can parse through, iterate over, and query spatial information using ArcPy.

In order to complete this question, you will need to have worked through Exercise 7 of your textbook. If you do not have the book, the required exercise and data can be found on our class' Canvas site (not github, there are copyright reasons for this). It will also help if you have read Chapter 8 of the text, especially for the second half of this question.

Your boss has decided that all airports that are within 10,000 meters of a road are "at risk" and all other airpots are "safe".

Using ArcPy, add a new field to the roads’ attribute table, call it “Security.” For all airports within 10,000 meters of a road, set the value of “Security” to “at risk,” for all other airports, set it to “safe.”
Think about how you would do this in Arc. Remember every tool in Arc has an equivalent in ArcPy. There are several ways to approach this problem, but do not use the Model Builder (in the real world, you might, but for the purposes of this exercise, you need to type the code yourself).

Think about what cursors do and how to use them, as well as how to add a field and select something by location. _Remember to use the arcpy.Usage() to look up syntax._ 


In [ ]:
# NOTE: I was one of the lucky folks that couldn't get arcpy working 
# with anaconda, so none of the arcpy labs have been tested inside of this notebook

import arcpy
import sys
import traceback

try:

    # location of road and airport shapefiles
    dataFolder = "[***update this to filepath where shapefiles are stored***]"

    arcpy.env.overwriteOuput = True
    arcpy.env.Workspace = dataFolder

    # shapefiles, layers, & other vars
    airports = dataFolder + "airports.shp"
    airportsLayer = "airportsLayer"

    roads = dataFolder + "roads.shp"
    roadsLayer = "roadsLayer"

    securityField = "Security"

    # add Security field to roads
    arcpy.management.AddField(airports, securityField, "TEXT", "", "", 15, "", "NULLABLE")

    # make feature layer of roads & airports
    arcpy.management.MakeFeatureLayer(roads, roadsLayer)
    arcpy.management.MakeFeatureLayer(airports, airportsLayer)

    # perform select by location for 'at risk' airports
    arcpy.management.SelectLayerByLocation(airportsLayer, "WITHIN_A_DISTANCE_GEODESIC", roadsLayer, "10000 Meters", "NEW_SELECTION")

    # loop through selected features and update Security field to 'at risk'
    with arcpy.da.UpdateCursor(airportsLayer, securityField) as cursor:
        for row in cursor:
            row[0] = "at risk"
            cursor.updateRow(row)

    # redo selection before updating Security field for 'safe' airports 
    arcpy.management.SelectLayerByAttribute(airportsLayer, "NEW_SELECTION", "{0} = ''".format(arcpy.AddFieldDelimiters(airportsLayer, securityField)))

    # loop through selected features and update Security fild to 'safe'
    with arcpy.da.UpdateCursor(airportsLayer, securityField) as cursor:
        for row in cursor:
            row[0] = "safe"
            cursor.updateRow(row)

except arcpy.ExecuteError: 
    # Get the tool error messages 
    msgs = arcpy.GetMessages(2) 

    # Return tool error messages for use with a script tool or Python Window
    arcpy.AddError(msgs) 

    # Print tool error messages
    print(msgs)

except:
    # Get the traceback object
    tb = sys.exc_info()[2]
    tbinfo = traceback.format_tb(tb)[0]

    # Concatenate information together concerning the error into a message string
    pymsg = "PYTHON ERRORS:\nTraceback info:\n{0}\nError Info:\n{1}".format(tbinfo,str(sys.exc_info()[1]))
    msgs = "ArcPy ERRORS:\n{0}\n".format(arcpy.GetMessages(2))

    # Return python error messages for use in script tool or Python Window
    arcpy.AddError(pymsg)
    arcpy.AddError(msgs)

    # Print Python error messages
    print(pymsg)
    print(msgs)

### Question 2 - An Odd Request (part 2)

Now we know what airports are safe and which ones are at risk. Very useful.

But, our weird boss is being even more weird. He (because it is me) wants you to find all of the airports that are within 50 kilometers of the “at risk” airports. These airports need to be categorized as “medium risk.”

Further, all of the “at risk” and “medium risk” airports must be put into a new shapefile called “Alaska at risk.shp.” You and your boss both know shapefile names can’t have spaces in them, but he insists that you accept the name with spaces and correct for it in python (Here I mean that in your script you have something that accepts a string with spaces and prorammatically removes them - this may seem banal in the context of this exercise, but the ability to correct for errors will pay huge dividends down the road... _pun intended_).

The script should produce a .shp file with the name Alaska_At_Risk.shp (this shapefile can be stored locally, you do not need to do any uploading, etc.).

Just like when doing things manually in ArcGIS, there are multiple orders and approaches to this task. You can create a new shapefile first, then update it with your selection. You can run the selection first, etc.).
The goal here is to grant you the ability to spatially query information with python just as you have previously done in Arc itself. With this ability, you can automate daily desktop GIS tasks.


In [ ]:
import arcpy
import sys
import traceback
from string import maketrans

def charFixer(inString):
    '''replaces spaces and @!/\:*?"<>| chars with underscores in a string'''
    # set up translation table
    badChars = " @!/\\:*?\"<>|"
    goodChars = "".join(["_" for i in badChars]) # kinda hokey, but must be same length as badChars
    tranTable = maketrans(badChars, goodChars)

    # run translate
    outString = inString.translate(tranTable)
    return outString

try:
    # location of road and airport shapefiles
    dataFolder = "[update with filepath to folder where shapefiles are stored]"

    arcpy.env.overwriteOuput = True
    arcpy.env.Workspace = dataFolder

    # shapefiles, layers, & other vars
    airports = dataFolder + "airports.shp"
    airportsLayer = "airportsLayer"
    airportsAtRisk = "airportsAtRisk"
    
    field = "Security"
    
    desiredName = "Alaska At Risk.shp"

    # create layers
    arcpy.management.MakeFeatureLayer(airports, airportsLayer, "{0} <> 'at risk'".format(arcpy.AddFieldDelimiters(airports, field)))
    arcpy.management.MakeFeatureLayer(airports, airportsAtRisk, "{0} = 'at risk'".format(arcpy.AddFieldDelimiters(airports, field)))

    # select by location
    arcpy.management.SelectLayerByLocation(airportsLayer, "WITHIN_A_DISTANCE_GEODESIC", airportsAtRisk, "50 Kilometers", "NEW_SELECTION")

    # update security field for medium risk airports
    with arcpy.da.UpdateCursor(airportsLayer, field) as cursor:
        for row in cursor:
            row[0] = "medium risk"
            cursor.updateRow(row)

    # create new shapefile with 'at risk' and 'medium risk' airports
    arcpy.conversion.FeatureClassToFeatureClass(airports, dataFolder, charFixer(desiredName), "{0} like '%% risk'".format(arcpy.AddFieldDelimiters(airports, field)))

except arcpy.ExecuteError: 
    # Get the tool error messages 
    msgs = arcpy.GetMessages(2) 

    # Return tool error messages for use with a script tool or Python Window
    arcpy.AddError(msgs) 

    # Print tool error messages
    print(msgs)

except:
    # Get the traceback object
    tb = sys.exc_info()[2]
    tbinfo = traceback.format_tb(tb)[0]

    # Concatenate information together concerning the error into a message string
    pymsg = "PYTHON ERRORS:\nTraceback info:\n{0}\nError Info:\n{1}".format(tbinfo,str(sys.exc_info()[1]))
    msgs = "ArcPy ERRORS:\n{0}\n".format(arcpy.GetMessages(2))

    # Return python error messages for use in script tool or Python Window
    arcpy.AddError(pymsg)
    arcpy.AddError(msgs)

    # Print Python error messages
    print(pymsg)
    print(msgs)

### Question 3 - Remember the Turtles

I’m sure you remember our friends the turtles. Well, in this exercise, you are going to use turtles to generate a series of points, and import those points as a polygon into ArcGIS. I know it might seem silly to use the turtle drawing module to create a polygon for Arc, but the point is to force you to think about how variously formatted data might be parsed into something ArcGIS can read and projected.

**It is strongly recommended that you work through the exercises for Chapter 8 here, particularly challenge problem 1.** 

Here's what you need to do (either in the next cell or as a standalone script):
1. Asks the user how many sides they would like their polygon to have. (Hint: use your script from lab 2).
2. Create said polygon using turtles, make sure you report the points (if you don’t know how, take a look at a list of turtle commands [here](https://docs.python.org/2/library/turtle.html).
3. Now you need to create a new feature class. Make sure you set your spatial reference. Use WGS1984. If you need a review of how to set spatial referents, check out pages 107-110 of your textbook.



In [ ]:
import arcpy
import turtle
import math
import sys
import traceback

# output vars, setup to create in a file geodatabase
# (because shapefiles can go to hell!!!)
outpath = "[path to geodatabase]"
outname = "turtle_poly"

def checkValue(val):
    '''check user input is valid'''
    # Note: large values *do* work, but go way off screen, 
    # since a fixed length is used for the sides
    if val.isdigit(): # check it can be changed to int
        if 0 < int(val) <= 360: # check it's positive and not too large
            return True
        else:
            return False
    else:
        return False

def apothem(s, n):
    '''get apothem based on length and number of sides'''
    a = s/(2*math.tan(math.radians(180.0/n)))
    return a

def createPoly(coords, outpath, outname):
    '''create polygon from list of coordinates'''
    arcpy.env.overwriteOutput = True
    spatialRef = arcpy.SpatialReference(4326) # WGS_1984 WKID
    vertices = []
    vertices.append(
        arcpy.Polygon(
           arcpy.Array([arcpy.Point(*tup) for tup in coords]), 
           spatialRef
           )
        )
        
    # create polygon
    arcpy.management.CopyFeatures(vertices, outpath + "/" + outname)
    print "\nFeature created. Click turtle screen to exit."

def drawNSides(sides):
    '''draw shape with N sides and return coords'''
    angle = 360.0/sides
    sideLen = 50
    apoth = apothem(sideLen, sides)
    startingPos = (-0.5*sideLen, -1*apoth) # Offset to center the polygon. Sort of...
    
    # setup turtle
    alex = turtle.Turtle()
    alex.shape("turtle")
    alex.color("yellow")
    alex.speed("fastest")

    #move to start position
    alex.penup()
    alex.setpos(startingPos)
    alex.pendown()
    
    # loop to create each side
    alex.forward(sideLen) # create first segment then enter loop
    alex.begin_poly()
    i = 1 # start at 1 since first segment is already created
    while i < sides:
        alex.left(angle)
        alex.forward(sideLen)    
        i += 1
    
    alex.end_poly()
    
    polyPoints = alex.get_poly()
    return polyPoints
    
# main program with recursive loop
def drawShapeMain(out_path, out_name):
    try:
        if arcpy.Exists(out_path):
            # get number of sides from user
            get_sides = input("\nDraw a shape with 'N' sides, where 'N' is a positive integer (360 max)." +
                              "\nHow many sides do you want? ") 
            
            # run check
            check = checkValue(str(get_sides))
            
            # draw if check passes, recurse if not
            if check == True:
                
                # setup screen
                wn = turtle.Screen()
                wn.bgcolor("purple")
    
                # draw polygon and return coords
                vertices = drawNSides(int(get_sides))
                
                # use coords to create polygon in 
                createPoly(vertices, out_path, out_name)

                # Wait for user to close window.
                # Note: doesn't seem to exit gracefully in jupyter, 
                # must restart kernal before running again   
                wn.exitonclick()

            else:
                BadInput = "\nInvalid value entered."
                raise BadInput

        else:
            raise ValueError("\nOutput path does not exist. Exiting.")

    except ValueError as err:
        print(err)

    except arcpy.ExecuteError: 
        # Get the tool error messages 
        msgs = arcpy.GetMessages(2) 

        # Return tool error messages for use with a script tool or Python Window
        arcpy.AddError(msgs) 

        # Print tool error messages
        print(msgs)

    except:
        # Get the traceback object
        tb = sys.exc_info()[2]
        tbinfo = traceback.format_tb(tb)[0]

        # Concatenate information together concerning the error into a message string
        pymsg = "PYTHON ERRORS:\nTraceback info:\n{0}\nError Info:\n{1}".format(tbinfo,str(sys.exc_info()[1]))
        msgs = "ArcPy ERRORS:\n{0}\n".format(arcpy.GetMessages(2))

        # Return python error messages for use in script tool or Python Window
        arcpy.AddError(pymsg)
        arcpy.AddError(msgs)

        # Print Python error messages
        print(pymsg)
        print(msgs)

### run the program ###
drawShapeMain(outpath, outname)


#### Challenge Problem 1 (and only) +1%

Can you make a script that asks the user how many polygons they would like, draws them sequentially (you can separate them by an arbitrary number of pixels, or create multiple turtles around the screen and draw them at once), and saves them as a multipart polygon? (There will be some hints on how to do this in your next class).


In [1]:
import arcpy
import turtle

arcpy.env.overwriteOutput = True

# output info
outpath = "[Path to geodatabase]"
outname = "turtle_multipoly"

# get 
polyCount = input("How many polygons do you want? ")

# turtle screen
scrn = turtle.Screen()
scrn.setworldcoordinates(-5,-5, 275, 275)
scrn.bgcolor("purple")
#scrn.tracer()

# polygon vars
sideLen = 50 
startX = 0
startY = 0
angle = 90

# assemble the troops. Create a turtle for each polygon
turtleSkwad = []

for n in range(polyCount):
    # setup turtle
    trooper = turtle.Turtle()
    trooper.shape("turtle")
    trooper.color("yellow")
    trooper.speed("fast")
    
    # set start position
    trooper.penup()
    trooper.setpos((startX, startY))
    
    # add turtle to list
    turtleSkwad.append(trooper)
    
    # go to new row after 5th poly
    if (n+1)%5 == 0 and startX != 0:
        # reset column
        startX = 0
        # advance row
        startY += (sideLen + 5)
    else:
        # advance column
        startX += (sideLen + 5)

# have the new recruits do their patrols and report their locations
multiPartArray = arcpy.Array()
spatialRef = arcpy.SpatialReference(4326) # WGS_1984 WKID

for recruit in turtleSkwad:
    # array to hold pooints for polygon
    singleArray = arcpy.Array()
    
    # grap starting coords
    singleArray.append(arcpy.Point(*recruit.pos()))
    
    # draw the polygon
    recruit.pendown()
    for i in range(4):
        recruit.forward(sideLen)
        recruit.left(angle)
        # add new coords to the array
        singleArray.append(arcpy.Point(*recruit.pos()))
        
    # create polygon class from array and add to the multi part array
    multiPartArray.append(singleArray)

multiPartPoly = arcpy.Polygon(multiPartArray, spatialRef)

# push the multi part polygon into a feature class
arcpy.management.CopyFeatures(multiPartPoly, outpath + "/" + outname)

print("Multipart polygon created. Click turtle screen to exit.")
scrn.exitonclick()


How many polygons do you want? 10
('n: ', 0)
('setpos', 0, 0)
('else', 55, 0)
('n: ', 1)
('setpos', 55, 0)
('else', 110, 0)
('n: ', 2)
('setpos', 110, 0)
('else', 165, 0)
('n: ', 3)
('setpos', 165, 0)
('else', 220, 0)
('n: ', 4)
('setpos', 220, 0)
('if', 0, 55)
('n: ', 5)
('setpos', 0, 55)
('else', 55, 55)
('n: ', 6)
('setpos', 55, 55)
('else', 110, 55)
('n: ', 7)
('setpos', 110, 55)
('else', 165, 55)
('n: ', 8)
('setpos', 165, 55)
('else', 220, 55)
('n: ', 9)
('setpos', 220, 55)
('if', 0, 110)


# HOUSEKEEPING!

The next lab will require you to be able to write scripts in both python 2.7 and python 3+. 
In anaconda, you can control the installed python version of a given environment when you create it. [See the cheat sheet](https://conda.io/docs/_downloads/conda-cheatsheet.pdf) for a reminder as to how.

**But, in addition**, due to some security features, you will also need to install a library called nb_conda_kernels. Once you are within the actived virtual environment, you can do so with the following command:
```
conda install --channel=conda-forge nb_conda_kernels 
```

This will allow you to select the version of python (kernel) when you load/create your notebook.

If you are writing your scripts in a text editor, you need not worry about this. Within the text editor, your code is simply text. You will simply have to make sure you run your scripts through interperters of the proper version.

## I am going to begin class assuming you are able to write/launch scripts in both python 2 and 3.

In [ ]:
#Here's an easy way to test your python version

print 'hello'

#If that command works, you are in python 2+; if it does not, you are in python 3+ (there are obviously other ways to do this)